# RAG 프로세스
## RAG 준비단계
![preprocess](./image/rag-1.png)
1. 문서 로드(Document Loader): pdf, 웹 문서 등으로부터 문서 내용을 불러온다
1. 분할(Splitter): 문서를 작은 chunk 단위로 분할한다
1. 임베딩(Embedding): 분할한 chunk를 벡터표현으로 변환한다
1. 벡터DB(Vector Store): 임베딩 chunk들을 벡터 DB에 저장한다
## RAG 단계
![rag](./image/rag-2.png)
1. 검색(Retriever): Query를 바탕으로 벡터DB에서 문서를 검색하기 위한 retriever 정의. retriever는 검색 알고리즘으로, dense와 sparse retriever가 있으며, dense는 유사도 기반 검색이고 sparse는 키워드 기반 검색이다.
1. 프롬프트(Prompt): RAG를 수행하기 위해 Prompt를 작성한다. Prompt context에 검색한 문서들이 들어가며, Prompt Engineering을 통해 답변 형식을 지정할 수 있다
1. LLM: LLM 모델 선정(GPT-4o, Claude3 Opus 등)
1. Chain: 위의 모든 단계들을 chain으로 정의하여 최종 output을 얻어낸다 

In [7]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [6]:
# 1단계: PDF 파일 로드
loader = PyMuPDFLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 2단계: Text Splitter 정의 및 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splitted_docs = text_splitter.split_documents(docs)

# 3단계: 임베딩 정의
embedding = HuggingFaceEmbeddings()

# 4단계: 벡터DB 생성 및 chunk 임베딩 저장
vectorstore = FAISS.from_documents(documents=splitted_docs, embedding=embedding)


In [13]:
# 5단계: retriever 생성
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=2048,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7049fd89d610>
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
# 체인 실행(Run Chain)
question = "Summarize as bullet point"
response = chain.invoke(question)
print(response)

**Summary in bullet points:**

* 구글은 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 구글과 클라우드 서비스 사용 계약도 체결
* 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 앤스로픽 및 오픈AI와 협력을 확대하는 추세
* 알리바바 클라우드는 최신 LLM ‘통이치엔원 2.0’을 공개하고, 산업별로 특화된 생성 AI 모델을 공개하는 한편, 모델 개발과 애플리케이션 구축 절차를 간소화하는 올인원 AI 모델 구축 플랫폼도 출시
* 빌 게이츠는 AI 에이전트가 5년 내 일상언어로 모든 작업을 처리할 수 있는 AI 에이전트가 보급되며 컴퓨터를 사용하는 방식이 완전히 바뀔 것으로 예상
* AI 에이전트는 의료와 교육, 생산성, 엔터테인먼트·쇼핑 영역의 서비스 대중화를 주도할 것

**Note:** I don't know the answer if it's not mentioned in the provided context.


In [15]:
print(chain.invoke("What is the name of AI developed by Samsung?"))

삼성 가우스(Samsung Gauss)입니다.
